# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-17.55,100,100,2.58,RU,1698368436
1,1,adamstown,-25.0660,-130.1015,22.86,77,65,3.11,PN,1698368437
2,2,waitangi,-43.9535,-176.5597,13.79,76,100,7.60,NZ,1698368437
3,3,emerald,-23.5333,148.1667,24.08,13,0,4.89,AU,1698368438
4,4,badger,64.8000,-147.5333,-0.57,84,40,0.00,US,1698368439


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height =600,
    size = "Humidity",
    color = "City",
    alpha= 0.75
)


# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
vacay_city_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] <4.5) & (city_data_df['Cloudiness'] == 0)]


# Drop any rows with null values
vacay_city_df.dropna()

# Display sample data
vacay_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,cabo san lucas,22.8909,-109.9124,25.66,69,0,3.66,MX,1698368392
50,50,uttar char fasson,22.2332,90.7132,23.36,86,0,3.18,BD,1698368447
52,52,as saffaniyah,27.9708,48.7300,22.06,52,0,3.57,SA,1698368447
82,82,shingu,33.7333,135.9833,22.49,46,0,2.33,JP,1698368453
88,88,zouerate,22.7187,-12.4521,24.34,24,0,3.58,MR,1698368454
149,149,topolobampo,25.6000,-109.0500,25.48,63,0,4.07,MX,1698368464
157,157,alamos,27.0167,-108.9333,22.29,51,0,0.91,MX,1698368400
173,173,illizi,26.4833,8.4667,21.89,28,0,2.51,DZ,1698368468
177,177,murzuq,25.9155,13.9184,23.78,30,0,2.66,LY,1698368468
199,199,dwarka,22.2394,68.9678,26.50,72,0,4.01,IN,1698368472


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy fuction to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacay_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\hveng\AppData\Local\Temp\ipykernel_9800\1660099658.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
30,cabo san lucas,MX,22.8909,-109.9124,69,
50,uttar char fasson,BD,22.2332,90.7132,86,
52,as saffaniyah,SA,27.9708,48.7300,52,
82,shingu,JP,33.7333,135.9833,46,
88,zouerate,MR,22.7187,-12.4521,24,
149,topolobampo,MX,25.6000,-109.0500,63,
157,alamos,MX,27.0167,-108.9333,51,
173,illizi,DZ,26.4833,8.4667,28,
177,murzuq,LY,25.9155,13.9184,30,
199,dwarka,IN,22.2394,68.9678,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row["Lat"]
    Longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
uttar char fasson - nearest hotel: No hotel found
as saffaniyah - nearest hotel: No hotel found
shingu - nearest hotel: ホテルニューパレス
zouerate - nearest hotel: فندق تيرس زمور
topolobampo - nearest hotel: Hotel Marina Topolobampo
alamos - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
murzuq - nearest hotel: فندق باريس - مرزق
dwarka - nearest hotel: The Dwarika Hotel
waddan - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
sagauli - nearest hotel: Utsav Kunj Hotel & Marriage Hall
huntsville - nearest hotel: 106 Jefferson Huntsville, Curio Collection by Hilton
medina - nearest hotel: The Oberoi Madina
matoury - nearest hotel: La Marmotte
sukumo - nearest hotel: 上村ビジネスホテル
rye - nearest hotel: Marriott Renaissance
saint-pierre - nearest hotel: Tropic Hotel
vertientes - nearest hotel: Hotel Rancho Hatuey
tindouf - nearest hotel: محمد بوسبي
new kingman-bu

,City,Country,Lat,Lng,Humidity,Hotel Name
30,cabo san lucas,MX,22.8909,-109.9124,69,Comfort Rooms
50,uttar char fasson,BD,22.2332,90.7132,86,No hotel found
52,as saffaniyah,SA,27.9708,48.7300,52,No hotel found
82,shingu,JP,33.7333,135.9833,46,ホテルニューパレス
88,zouerate,MR,22.7187,-12.4521,24,فندق تيرس زمور
149,topolobampo,MX,25.6000,-109.0500,63,Hotel Marina Topolobampo
157,alamos,MX,27.0167,-108.9333,51,No hotel found
173,illizi,DZ,26.4833,8.4667,28,بوناقة
177,murzuq,LY,25.9155,13.9184,30,فندق باريس - مرزق
199,dwarka,IN,22.2394,68.9678,72,The Dwarika Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
Vacation_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height =600,
    size = "Humidity",
    color = "City",
    alpha= 0.7,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
Vacation_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)